<a href="https://colab.research.google.com/github/Niraj-Khatri/Pyspark-AWS/blob/master/Electronics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Spark and Import Dataset

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
# Connect to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-02 19:01:27--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-09-02 19:01:27 (10.4 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Import Amazon Electronics Data File From AWS


In [ ]:
# Import struct fields
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, TimestampType

In [ ]:


# Create schema of field types
schema = [StructField("marketplace", StringType(), True), StructField("customer_id", StringType(), True), StructField("review_id", StringType(), True), 
          StructField("product_id", StringType(), True), StructField("product_parent", StringType(), True),
          StructField("product_title", StringType(), True), StructField("product_category", StringType(), True), StructField("star_rating", IntegerType(), True),
          StructField("helpful_votes", IntegerType(), True), StructField("total_votes", IntegerType(), True), StructField("vine", StringType(), True),
          StructField("verified_purchase", StringType(), True), StructField("review_headline", StringType(), True), StructField("review_date", StringType(), False)]

final = StructType(fields=schema)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
data = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep='\t', header=True)

# Clean the data
data = data.withColumn('review_date', data['review_date'].cast(TimestampType()))
data = data.withColumn('customer_id', data['customer_id'].cast(IntegerType()))
data = data.withColumn('product_parent', data['product_parent'].cast(IntegerType()))
data = data.withColumn('star_rating', data['star_rating'].cast(IntegerType()))
data = data.withColumn('helpful_votes', data['helpful_votes'].cast(IntegerType()))
data = data.withColumn('total_votes', data['total_votes'].cast(IntegerType()))
df = data.dropna()

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.|2015-08-31 00:00:00|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...| 

# Create Tables

## Customers Table

In [ ]:
# Customer ID with customer view count
customers = df.groupBy("customer_id").count()
customers = customers.withColumnRenamed('count','customer_count')
customers = customers.withColumn('customer_count', customers['customer_count'].cast(IntegerType()))


## Products Table


In [ ]:
# Product ID and product title
products = df.groupby("product_id", "product_title").count().select("product_id", "product_title")


## Reviews Table


In [ ]:
# The review number, who reviewed it, what he/she reviewed, the parent of the reviewed product, and the date of the review
reviews = df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")

## Vines Table

In [ ]:
# The review number, rating, helpful votes, total votes, and vine 
vines = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")

# Connect To RDS

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonreviews.c5bki1uyc90h.us-east-2.rds.amazonaws.com:5432/AmazonReviews"
config = {"user":"postgres", 
          "password": "Goldensun123!", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write tables to postgres in RDS
customers.write.jdbc(url=jdbc_url, table = "customers", mode=mode, properties=config)
products.write.jdbc(url=jdbc_url, table = "products", mode=mode, properties=config)
reviews.write.jdbc(url=jdbc_url, table = "review_id_table", mode=mode, properties=config)
vines.write.jdbc(url=jdbc_url, table = "vine_table", mode=mode, properties=config)
